Importações e configuração

In [15]:
import pandas as pd
from pathlib import Path

# Caminhos base dentro do container Jupyter
BASE_RAW = Path("/home/jovyan/data/raw")
BASE_PROC = Path("/home/jovyan/data/processed")

# Garante que a pasta processed existe
BASE_PROC.mkdir(parents=True, exist_ok=True)

BASE_RAW, BASE_PROC

(PosixPath('/home/jovyan/data/raw'), PosixPath('/home/jovyan/data/processed'))

Processamento

In [16]:
def processar_inmet(path: Path) -> pd.DataFrame:

    print(f"\n📄 Lendo arquivo: {path.name}")
    
    # 1) Ler CSV bruto
    df = pd.read_csv(
        path,
        sep=';',
        skiprows=8,           # pula bloco de metadados
        encoding='latin1',
        low_memory=False
    )
    
    # 2) Renomear colunas de interesse (só as que existirem)
    mapa_renome = {
        'Data': 'data',
        'Hora UTC': 'hora',
        'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)': 'temp_ar',
        'UMIDADE RELATIVA DO AR, HORARIA (%)': 'umidade',
        'RADIACAO GLOBAL (Kj/m²)': 'radiacao',
        'RADIACAO GLOBAL (kJ/m²)': 'radiacao',
        'RADIACAO GLOBAL': 'radiacao',
        'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)': 'precipitacao',
        'VENTO, VELOCIDADE HORARIA (m/s)': 'vento_vel',
        'VENTO, DIREÇÃO HORARIA (gr) (° (gr))': 'vento_dir',
        'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)': 'pressao'
    }
    df = df.rename(columns={orig: novo for orig, novo in mapa_renome.items() if orig in df.columns})

    # 3) Garante que data e hora existem
    if 'data' not in df.columns or 'hora' not in df.columns:
        raise ValueError("Colunas 'Data' e/ou 'Hora UTC' não foram encontradas após o rename.")

    # 4) Padronizar 'data' e 'hora' como string
    df['data'] = df['data'].astype(str).str.strip()
    df['hora'] = df['hora'].astype(str).str.strip()

    # 5) Remover ' UTC' da hora, se existir
    df['hora'] = df['hora'].str.replace(' UTC', '', regex=False)

    # 6) Se hora estiver como 0, 300, 1200 etc, padronizar para HH:MM
    mascara_numerica = df['hora'].str.fullmatch(r'\d{1,4}')
    df.loc[mascara_numerica, 'hora'] = (
        df.loc[mascara_numerica, 'hora']
          .str.zfill(4)
          .str[:2] + ':' + df.loc[mascara_numerica, 'hora'].str.zfill(4).str[2:]
    )

    # 7) Criar coluna datetime (deixa o pandas inferir formato)
    df['datetime'] = pd.to_datetime(
        df['data'] + ' ' + df['hora'],
        errors='coerce'
    )

    # Remover linhas onde datetime não pôde ser montado
    df = df.dropna(subset=['datetime'])
    df = df.set_index('datetime')

    # 8) Selecionar colunas de interesse (as que existirem)
    colunas_desejadas = ['hora', 'temp_ar', 'umidade', 'radiacao',
                         'vento_vel', 'precipitacao', 'pressao']
    colunas_presentes = [c for c in colunas_desejadas if c in df.columns]
    df = df[colunas_presentes]

    # 9) Converter variáveis numéricas para float
    variaveis_numericas = [c for c in colunas_presentes if c != 'hora']

    for col in variaveis_numericas:
        df[col] = df[col].astype(str).str.replace(',', '.', regex=False).str.strip()
        df[col] = pd.to_numeric(df[col], errors='coerce')

    # 10) Remover colunas que estão 100% NaN (por exemplo, radiacao ausente o ano todo)
    colunas_todas_nan = [c for c in variaveis_numericas if df[c].isna().sum() == len(df)]
    if colunas_todas_nan:
        print("   ⚠ Removendo colunas 100% NaN:", colunas_todas_nan)
        df = df.drop(columns=colunas_todas_nan)
        variaveis_numericas = [c for c in variaveis_numericas if c not in colunas_todas_nan]

    # 11) Interpolar valores faltantes ao longo do tempo
    if variaveis_numericas:
        df[variaveis_numericas] = df[variaveis_numericas].interpolate(
            method='time'
        )

        # Opcional: preencher pontas (início/fim) com ffill/bfill
        df[variaveis_numericas] = df[variaveis_numericas].ffill().bfill()

    # 12) Remover apenas linhas em que TODAS as variáveis numéricas são NaN (caso sobrem)
    if variaveis_numericas:
        mask_all_nan = df[variaveis_numericas].isna().all(axis=1)
        n_all_nan = mask_all_nan.sum()
        if n_all_nan > 0:
            print(f"   ⚠ Removendo {n_all_nan} linhas 100% NaN nas variáveis numéricas.")
            df = df[~mask_all_nan]

    # 13) Criar features auxiliares
    df['hora_num'] = df.index.hour
    df['mes'] = df.index.month

    print(f"   ✅ Registros após tratamento: {len(df)}")
    return df

Loop para processar Petrolina e Garanhuns (2020 a 2024)

In [17]:
anos = ['2020', '2021', '2022', '2023', '2024']
cidades = ['PETROLINA', 'GARANHUNS']

for ano in anos:
    pasta_ano = BASE_RAW / ano
    
    for cidade in cidades:
        # procura arquivos do tipo *CIDADE*.CSV dentro do ano
        arquivos = list(pasta_ano.glob(f"*{cidade}*.CSV"))
        
        if not arquivos:
            print(f"\n🚫 Nenhum arquivo encontrado para {cidade} em {ano}")
            continue
        
        for arq in arquivos:
            df_tratado = processar_inmet(arq)
            
            # nome do arquivo de saída
            nome_saida = f"{cidade.lower()}_{ano}_tratado.csv"
            caminho_saida = BASE_PROC / nome_saida
            
            df_tratado.to_csv(caminho_saida)
            print(f"   💾 Salvo em: {caminho_saida}")


📄 Lendo arquivo: INMET_NE_PE_A307_PETROLINA_01-01-2020_A_31-12-2020.CSV
   ✅ Registros após tratamento: 8784
   💾 Salvo em: /home/jovyan/data/processed/petrolina_2020_tratado.csv

📄 Lendo arquivo: INMET_NE_PE_A322_GARANHUNS_01-01-2020_A_31-12-2020.CSV
   ✅ Registros após tratamento: 8784
   💾 Salvo em: /home/jovyan/data/processed/garanhuns_2020_tratado.csv

📄 Lendo arquivo: INMET_NE_PE_A307_PETROLINA_01-01-2021_A_31-12-2021.CSV
   ✅ Registros após tratamento: 8760
   💾 Salvo em: /home/jovyan/data/processed/petrolina_2021_tratado.csv

📄 Lendo arquivo: INMET_NE_PE_A322_GARANHUNS_01-01-2021_A_31-12-2021.CSV
   ✅ Registros após tratamento: 8760
   💾 Salvo em: /home/jovyan/data/processed/garanhuns_2021_tratado.csv

📄 Lendo arquivo: INMET_NE_PE_A307_PETROLINA_01-01-2022_A_31-12-2022.CSV
   ✅ Registros após tratamento: 8760
   💾 Salvo em: /home/jovyan/data/processed/petrolina_2022_tratado.csv

📄 Lendo arquivo: INMET_NE_PE_A322_GARANHUNS_01-01-2022_A_31-12-2022.CSV
   ✅ Registros após tratame

Resumo

In [18]:
processed_path = BASE_PROC

arquivos_tratados = list(processed_path.glob("*_tratado.csv"))

print("Foram encontrados", len(arquivos_tratados), "arquivos tratados.\n")

for arquivo in arquivos_tratados:
    df = pd.read_csv(arquivo, index_col=0, parse_dates=True)
    
    print("📄 Arquivo:", arquivo.name)
    print("   ➤ Linhas:", len(df))
    print("   ➤ NaNs por coluna:")
    print(df.isna().sum().to_string())
    print("-" * 60)


Foram encontrados 10 arquivos tratados.

📄 Arquivo: petrolina_2020_tratado.csv
   ➤ Linhas: 8784
   ➤ NaNs por coluna:
hora            0
temp_ar         0
umidade         0
radiacao        0
vento_vel       0
precipitacao    0
pressao         0
hora_num        0
mes             0
------------------------------------------------------------
📄 Arquivo: petrolina_2022_tratado.csv
   ➤ Linhas: 8760
   ➤ NaNs por coluna:
hora            0
temp_ar         0
umidade         0
radiacao        0
vento_vel       0
precipitacao    0
pressao         0
hora_num        0
mes             0
------------------------------------------------------------
📄 Arquivo: garanhuns_2024_tratado.csv
   ➤ Linhas: 8784
   ➤ NaNs por coluna:
hora         0
temp_ar      0
umidade      0
radiacao     0
vento_vel    0
pressao      0
hora_num     0
mes          0
------------------------------------------------------------
📄 Arquivo: garanhuns_2023_tratado.csv
   ➤ Linhas: 8760
   ➤ NaNs por coluna:
hora            0
te

Nota:

Para lidar com valores climáticos faltantes nos arquivos do INMET, utilizamos a técnica de interpolação temporal, que é um método recomendado internacionalmente para reconstrução de séries meteorológicas. Em termos simples, a interpolação preenche lacunas usando os valores anteriores e posteriores como referência. Por exemplo: se às 10h a temperatura é 22°C e às 12h é 24°C, a interpolação estima que às 11h ela seria aproximadamente 23°C. Isso evita descartar horas inteiras — o que reduziria drasticamente a qualidade do conjunto de dados — e mantém a continuidade natural do clima, que muda de forma gradual ao longo do tempo. Esse procedimento foi aplicado usando df.interpolate(method='time'), que utiliza o índice de datas do próprio dataframe para estimar valores de forma coerente com a evolução temporal. O uso de interpolação em dados climáticos é amplamente adotado por instituições como o NOAA – National Oceanic and Atmospheric Administration, que afirma que a interpolação linear é adequada para preenchimento de falhas curtas em séries meteorológicas contínuas (NOAA, Climate Data Interpolation Guidelines, 2018).

Configuração SnowFlake

In [ ]:
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas
import os
import getpass

SNOWFLAKE_USER = "APP_USER_METEO"
SNOWFLAKE_PASSWORD = "B@tataQuente!2025" 
SNOWFLAKE_ACCOUNT = "gcifwsf-dz69699" 
SNOWFLAKE_WAREHOUSE = "WH_METEORO_BI"
SNOWFLAKE_DATABASE = "DB_INMET_PE"
SNOWFLAKE_SCHEMA = "DATA_ESTRUTURADA"
SNOWFLAKE_TABLE = "DADOS_INMET_HORARIO"

# Cria a string de conexão para uso em funções
SNOWFLAKE_CONN_PARAMS = {
    "user": SNOWFLAKE_USER,
    "password": SNOWFLAKE_PASSWORD,
    "account": SNOWFLAKE_ACCOUNT,
    "warehouse": SNOWFLAKE_WAREHOUSE,
    "database": SNOWFLAKE_DATABASE,
    "schema": SNOWFLAKE_SCHEMA
}

print("Configurações do Snowflake definidas.")

Configurações do Snowflake definidas.


Função para carregar dados no SnowFlake

In [ ]:
# ==============================================================================
# FUNÇÃO PARA CARREGAR DADOS NO SNOWFLAKE
# ==============================================================================

def carregar_para_snowflake(df: pd.DataFrame, nome_tabela: str):
    """
    Conecta ao Snowflake e carrega um DataFrame Pandas para a tabela especificada.
    """
    print(f"\n❄️ Conectando ao Snowflake para carregar a tabela {nome_tabela}...")
    
    try:
        # 1. Conexão
        conn = snowflake.connector.connect(**SNOWFLAKE_CONN_PARAMS)
        
        # 2. Executa a carga usando write_pandas
        success, n_chunks, n_rows, output = write_pandas(
            conn, 
            df, 
            table_name=nome_tabela,
            database=SNOWFLAKE_DATABASE,
            schema=SNOWFLAKE_SCHEMA,
            auto_create_table=False 
        )
        
        # 3. Confirmação
        print(f"   ✅ Carga concluída. Linhas carregadas: {n_rows}")
        print(f"   ✅ Arquivos enviados: {n_chunks} chunk(s).")
        
    except Exception as e:
        print(f"   ❌ ERRO ao carregar para o Snowflake: {e}")
        print("   Primeiras linhas do DataFrame com erro:")
        print(df.head())
        
    finally:
        if 'conn' in locals() and conn:
            conn.close()

Loop para processar, salvar CSV localmente e carregar no SnowFlake

In [ ]:
# ==============================================================================
# LOOP PARA PROCESSAR, SALVAR CSV LOCALMENTE E CARREGAR NO SNOWFLAKE
# ==============================================================================

try:
    with snowflake.connector.connect(**SNOWFLAKE_CONN_PARAMS) as conn:
        conn.cursor().execute(f"TRUNCATE TABLE {SNOWFLAKE_DATABASE}.{SNOWFLAKE_SCHEMA}.{SNOWFLAKE_TABLE}")
        print(f"\n⚠️ Tabela {SNOWFLAKE_TABLE} truncada com sucesso no Snowflake.")
except Exception as e:
    print(f"\n❌ ERRO ao tentar truncar a tabela no Snowflake: {e}")

for ano in anos:
    pasta_ano = BASE_RAW / ano
    
    for cidade in cidades:
        estacao_id = cidade.lower()
        
        arquivos = list(pasta_ano.glob(f"*{cidade}*.CSV"))
        
        if not arquivos:
            print(f"\n🚫 Nenhum arquivo encontrado para {cidade} em {ano}")
            continue
        
        for arq in arquivos:
            # 1. Processamento e Limpeza (Função existente)
            df_tratado = processar_inmet(arq)
            
            # 2. Ajustes Finais para o Snowflake
            
            # 2a. Adiciona a coluna ESTACAO_ID
            df_tratado['ESTACAO_ID'] = estacao_id
            
            # 2b. Renomeia colunas para bater com o DDL final
            # Colunas no DDL: TEMP_AR, UMIDADE, PRESSAO, VENTO_VEL, RADIACAO, PRECIPITACAO
            mapa_cols_snowflake = {
                'pressao': 'PRESSAO',
                'radiacao': 'RADIACAO',
                'vento_vel': 'VENTO_VEL',
                'precipitacao': 'PRECIPITACAO',
                'temp_ar': 'TEMP_AR',
                'umidade': 'UMIDADE',
                'hora_num': 'HORA_NUM',
                'mes': 'MES',
                'hora': 'HORA'
            }

            df_final = df_tratado.reset_index().rename(columns=mapa_cols_snowflake)
        
            if 'index' in df_final.columns:
                df_final = df_final.drop(columns=['index'])

            df_final.rename(columns={'datetime': 'DATETIME'}, inplace=True)
            
            # Reorganiza as colunas na ordem do DDL do Snowflake
            colunas_ddl = [
                'DATETIME', 'ESTACAO_ID', 'TEMP_AR', 'UMIDADE', 'PRESSAO', 
                'VENTO_VEL', 'RADIACAO', 'PRECIPITACAO', 'HORA_NUM', 'MES', 'HORA'
            ]
            
            # Filtra colunas que realmente existem no DataFrame (ex: 'RADIACAO' pode ter sido removida)
            colunas_existentes = [c for c in colunas_ddl if c in df_final.columns]
            df_final = df_final[colunas_existentes]


            # 3. Salva CSV Localmente 
            nome_saida = f"{estacao_id}_{ano}_tratado.csv"
            caminho_saida = BASE_PROC / nome_saida
            df_tratado.to_csv(caminho_saida)
            print(f"   💾 Salvo em: {caminho_saida}")
            
            # 4. Carga para o Snowflake
            carregar_para_snowflake(df_final, SNOWFLAKE_TABLE)


⚠️ Tabela DADOS_INMET_HORARIO truncada com sucesso no Snowflake.

📄 Lendo arquivo: INMET_NE_PE_A307_PETROLINA_01-01-2020_A_31-12-2020.CSV
   ✅ Registros após tratamento: 8784
   💾 Salvo em: /home/jovyan/data/processed/petrolina_2020_tratado.csv

❄️ Conectando ao Snowflake para carregar a tabela DADOS_INMET_HORARIO...


/opt/conda/lib/python3.11/site-packages/snowflake/connector/pandas_tools.py:407: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  [pandas.api.types.is_datetime64tz_dtype(df[c]) for c in df.columns]


   ✅ Carga concluída. Linhas carregadas: 8784
   ✅ Arquivos enviados: 1 chunk(s).

📄 Lendo arquivo: INMET_NE_PE_A322_GARANHUNS_01-01-2020_A_31-12-2020.CSV
   ✅ Registros após tratamento: 8784
   💾 Salvo em: /home/jovyan/data/processed/garanhuns_2020_tratado.csv

❄️ Conectando ao Snowflake para carregar a tabela DADOS_INMET_HORARIO...


/opt/conda/lib/python3.11/site-packages/snowflake/connector/pandas_tools.py:407: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  [pandas.api.types.is_datetime64tz_dtype(df[c]) for c in df.columns]


   ✅ Carga concluída. Linhas carregadas: 8784
   ✅ Arquivos enviados: 1 chunk(s).

📄 Lendo arquivo: INMET_NE_PE_A307_PETROLINA_01-01-2021_A_31-12-2021.CSV
   ✅ Registros após tratamento: 8760
   💾 Salvo em: /home/jovyan/data/processed/petrolina_2021_tratado.csv

❄️ Conectando ao Snowflake para carregar a tabela DADOS_INMET_HORARIO...


...

   ✅ Carga concluída. Linhas carregadas: 8760
   ✅ Arquivos enviados: 1 chunk(s).

📄 Lendo arquivo: INMET_NE_PE_A307_PETROLINA_01-01-2024_A_31-12-2024.CSV
   ⚠ Removendo colunas 100% NaN: ['radiacao']
   ✅ Registros após tratamento: 8784
   💾 Salvo em: /home/jovyan/data/processed/petrolina_2024_tratado.csv

❄️ Conectando ao Snowflake para carregar a tabela DADOS_INMET_HORARIO...


/opt/conda/lib/python3.11/site-packages/snowflake/connector/pandas_tools.py:407: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  [pandas.api.types.is_datetime64tz_dtype(df[c]) for c in df.columns]


   ✅ Carga concluída. Linhas carregadas: 8784
   ✅ Arquivos enviados: 1 chunk(s).

📄 Lendo arquivo: INMET_NE_PE_A322_GARANHUNS_01-01-2024_A_31-12-2024.CSV
   ⚠ Removendo colunas 100% NaN: ['precipitacao']
   ✅ Registros após tratamento: 8784
   💾 Salvo em: /home/jovyan/data/processed/garanhuns_2024_tratado.csv

❄️ Conectando ao Snowflake para carregar a tabela DADOS_INMET_HORARIO...


/opt/conda/lib/python3.11/site-packages/snowflake/connector/pandas_tools.py:407: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  [pandas.api.types.is_datetime64tz_dtype(df[c]) for c in df.columns]


   ✅ Carga concluída. Linhas carregadas: 8784
   ✅ Arquivos enviados: 1 chunk(s).
